In [2]:
import requests
import json
from bs4 import BeautifulSoup as bs
import re

# REPLACE WITH OTHER POSTS!
path = 'https://blindgossip.com/photo-blind-item-47'


In [3]:
# includes common typos
celebs_nicknames = {
    'madonna' : ['madge', 'maddona'],
    'dean cain' : ['dean cane'],
    'kardashian': ['kardashians', 'jenner'],
    'jennifer lopez' : ['j lo'], 
    'meghan markle' : ['meghan', 'the markle'],
    'prince harry' : ['harry'],
    'harry connick jr' : ['harry connick', 'connick jr'],
    'johnny depp' : ['johnny dep'],
    'angelina jolie' : ['ange', 'angie jo'],
    'oprah winfrey' : ['oprah'],
    'Gwyneth Paltrow' : ['goop']
}

In [4]:
## /////// ##
## Utils ##
## /////// ##
def updateOrCreateKey(dictt, keyy, valuee):
    if keyy not in dictt:
        dictt[keyy] = [valuee]
    else:
        if isinstance(dictt[keyy], list):
            dictt[keyy].append(valuee)
        else:
            dictt[keyy] = [dictt[keyy], valuee]

first_words_to_remove = ['looks like', 'must be', 'this is', 'gotta be', 'is it', 'maybe',
                              'thats', 'its', 'that\'s', 'it\'s', '']
remove_from_possible_names = ['my god', 'this', 'omg', 'oops', 'yes', 'agreed', 'what', 'i\'m', 'ugh', 'lol']
indicative_first_words = ['looks like', 'must be', 'this is', 'gotta be', 'is it', 
                              'thats', 'its', 'that\'s', 'it\'s', 'i know'] #'maybe'? 

 
## /////// ##
## Scrape and Parse ##   
## /////// ##
def getDataFromPath(url):
    data = requests.get(url)
    if data.status_code == 200:
        return data.content, data.text
    else:
        print ("could not load post")
  
def getQuestion(data):
    try:
        soup = bs(data, 'html.parser')
        body = soup.findAll(class_ = 'entry-content')
        alltags = body[0].find_all('p')
        questions = []
        for i in alltags:
            if i.string:
                content = i.string
                # few words + ':' --> question, or starts with '[Optional]'
                if content.find(':')>-1 and content.count(' ')<3 and str(i.string).lower() not in ['clues:']:
                    questions.append(content.replace(':', ''))
                if content.find('[Optional]') >-1:
                    questions.append(content)  
        return questions
    except:
        print ('error while parsing the questions')

    
def getComments(data):    
    try:
        soup = bs(data, 'html.parser')
        body = soup.findAll(class_ = 'comment-list')
        comments = {}
        allcomments = body[0].findAll('li')
        for i in allcomments:
            commnt_name = i.findAll(class_ ='comment-author-name')
            name = commnt_name[0].string
            commnt_txt = i.findAll(class_ = 'comment-content')
            txt = '. '.join(commnt_txt[0].findAll(text=True))
            updateOrCreateKey(comments, name, txt)
        return comments
    except:
        print ('error while parsing the comments')

## /////// ##
## Clean comment ##   
## /////// ##
def cleanComment(comment):
    RE_SPACE_TO_REMOVE = re.compile(r'\s+')
    comment = RE_SPACE_TO_REMOVE.sub(' ', comment)
    comment = comment.replace('’', '\'')
    comment = comment.replace('–', '-')
    comment = comment.strip()
    return comment

## /////// ##
## Get possible names ##
## /////// ##
def cutCommentToStopper(comment, strt=0):
    new_comment = ''
    stoppers = set()
    stoppers.add(comment.find('.', strt))
    stoppers.add(comment.find(',', strt))
    #stoppers.add(comment.find('-', strt))  #can appear mid-name
    stoppers.add(comment.find('?', strt))
    stoppers.add(comment.find('!', strt))
    #stoppers.add(comment.find('–', strt))
    stoppers.add(comment.find('…', strt))
    stoppers.discard(-1)
    stopper = min(stoppers) if stoppers else len(comment) 
    new_comment = comment[strt:stopper].strip()
    return new_comment


def possibleNameShortSegment(comment):
    name = ''
    shorter = cutCommentToStopper(comment)
    # remove indicative first words
    if shorter:
        for i in first_words_to_remove:
            pattern = '\\b{}\\b'.format(i)
            regex = re.compile(pattern, re.IGNORECASE)
            location = regex.search(shorter)
            if location and location.start()==0:
                shorter = shorter[len(i):]
                break 
    if shorter:
        shorter = shorter.strip()
        # if 1-2 words: take as is
        if shorter.count(' ')<2:
            name = cleanBeforeAdding(shorter)
        # if 3-5 words: take upper case only (special logic for 1st word)
        elif shorter.count(' ')<6:
            nameDirty = getUpperCaseFromShort(shorter)
            name = cleanBeforeAdding(nameDirty) 
    return name

def getUpperCaseFromShort(short):
    cmnt_upper = []
    name = ''
    for i in short.split():
        word = i if i[0].isupper() else ''
        cmnt_upper.append(word) 
    if cmnt_upper:
        #if 2nd word is upper --> check 1st, take 1, 2, .. until no longer upper
        if len(cmnt_upper)>1 and cmnt_upper[1]:
            strt = 0 if cmnt_upper[0] else 1
            j=0
            name = ''
            while len(cmnt_upper)>(strt+j) and cmnt_upper[strt+j]:
                name = name + cmnt_upper[strt+j]+ ' '
                j+=1
        #if 2nd lower --> keep walking, if find upper take current, next, etc. until no longer
        # TODO- adjust for cmnt_upper length == 1 (celebrity name length 1 word, like Madonna)
        else:
            name = ''
            for index, element in enumerate(cmnt_upper[2:]):
                if element: 
                    j=0
                    while len(cmnt_upper)>(2+index+j) and cmnt_upper[2+index+j]:
                        name = name + cmnt_upper[2+index+j]+ ' '
                        j+=1
                    break
    return name.strip()

def getAfterQuestion(comment, questions):
    names = []
    for i in questions:
        i = 'Optional' if i.startswith('[Optional]') else i
        j = comment.lower().find(i.lower())
        # if question found in comment, cut from it until stopper and treat as short segment
        if j!=-1:
            shorter = cutCommentToStopper(comment, j+len(i)).strip()  
            if shorter.count(' ')>0:
                shorter = ' '.join(shorter.split()[:5])
                nameDirty = getUpperCaseFromShort(shorter)
                name = cleanBeforeAdding(nameDirty)
                if name: 
                    names.append(name)
            elif len(shorter)>2:
                name = cleanBeforeAdding(shorter)
                if name:
                    names.append(name)
    return names

def getUpperAfterKeyWrods(comment):
    names = []
    #key words may appear several times, take all occurrences
    for word in indicative_first_words:
        pattern = '\\b{}\\b'.format(word)
        regex = re.compile(pattern, re.IGNORECASE)
        matches = re.finditer(regex, comment)
        if matches:
            for match in matches:
                #get only next few words/less if stopper
                shorter = cutCommentToStopper(comment, match.start()).strip()
                shorter = ' '.join(shorter.split()[:5])
                if shorter:
                    shorter = shorter[len(word):].strip()
                    nameDirty = getUpperCaseFromShort(shorter)
                    name = cleanBeforeAdding(nameDirty) 
                    if name: 
                        names.append(name)
    return names

def getPossibleNames(comment):
    names = []
    comment = cleanComment(comment)  
    # get possible name based on first short segment
    name = possibleNameShortSegment(comment)
    if name: 
        names.append(name) 
        #print('short', name)
    # get possible name based on question ("Actor:")
    name = getAfterQuestion(comment, questions)
    if name:
        names.extend(name)
        #print ('question', name)
    # get possible name based on key word ("that's")
    name = getUpperAfterKeyWrods(comment)
    if name:
        names.extend(name)
        #print ('key', name)
    # set for cases when name was caught by 2 logic paths
    return list(set(names))


## /////// ##
## Clean up names in the possible names list ##
## /////// ##
def removeBadNames(name):
    return None if name in remove_from_possible_names else name


def removeQuestionFromName(name, questions):
    for i in questions:
        question = i.lower().strip()
        if name.lower().find(question)==0:
            name = name[len(question):].strip()
    return name
        

def cleanBeforeAdding(possiblename):
    possiblename.strip()
    possiblename = possiblename.lower()
    #possiblename = possiblename.replace('-', ' ')
    possiblename = possiblename.replace(':', ' ')
    possiblename = removeQuestionFromName(possiblename, questions)
    possiblename = removeBadNames(possiblename)
    # TODO?: comment = re.sub(r'[^a-zA-Z1-9\' ]', '', comment)
    return possiblename

## /////// ##
### Count results + merge ###
## /////// ##
def superCount(comments, all_comments):
    final_cntr = {}
    names = []
    counted_cmnts = {}
    for k, v in all_comments.items():
        for vv in v:
            names.append(vv)
    names = set(names)
    for k, v in comments.items():
        for vv in v:
            vv = cleanComment(vv)
            for name in names:
                if name.casefold() in vv.casefold():
                    if name in final_cntr:
                        final_cntr[name] = final_cntr[name]+1
                    else:
                        final_cntr[name] = 1
                    updateOrCreateKey(counted_cmnts, k, vv)
    return final_cntr, counted_cmnts

def healthCounter(all_comments):
    cntr = {}
    for k,v in all_comments.items():
        for keyy in v:
            if keyy not in cntr:
                cntr[keyy] = 1
            else:
                cntr[keyy] = cntr[keyy]+1
    return cntr

def mergeByNicknames(final_cntr, nicknames):
    nick_cntr = {}
    skip_keys = []
    for k,v in final_cntr.items():
        # if no nickname- take as is
        if k not in nicknames or k in skip_keys:
            nick_cntr[k] = v
        else:
            for nickname in nicknames[k]:
                # if nickname is a key, sum its value with the original name. If not- take the name+ value as is
                if nickname in final_cntr:
                    if k in nick_cntr:
                        nick_cntr[k] = nick_cntr[k] + final_cntr[nickname]
                    else:
                        nick_cntr[k] = final_cntr[k] + final_cntr[nickname]
                    skip_keys.append(nickname)
                else:
                    nick_cntr[k] = nick_cntr[k] if k in nick_cntr else v          
    # remvoe nicknames that were already counted
    for i in set(skip_keys):
        del nick_cntr[i]
    return nick_cntr


def print_results(nick_cntr, comments):
    # print top3 guesses as % of all guesses, but only if >1 person guessed
    all_names = sum(nick_cntr.values())
    results = {k:round((v/all_names)*100) for k, v in nick_cntr.items() if v>1}
    results = sorted(results.items(), key = lambda results: results[1], reverse = True)
    if len(results)==0:
        if len(comments)<20:
            print ("Sorry, couldn't figure this one out. Try again when there are more comments?")
        else: 
            print ("Sorry, couldn't figure this one out ⊙︿⊙")
    else:
        print("Top guesses are:")
        i=0
        while i<4 and i<len(results):
            name = results[i][0]
            print(name.title(), "("+str(results[i][1])+"%)")
            i+=1
            
       
## /////// ##
### RUN THE CODE ###
## /////// ##
# Scrape and Parse data (as global variables)
data, data_text = getDataFromPath(path)            
questions = getQuestion(data)

def main():    
    comments = getComments(data)
    all_comments = {}
    possible_names = []
    # get list of possible names
    for commenter, texts_list in comments.items():
        for element in texts_list: 
            #print(commenter, element)
            possible_names = getPossibleNames(element)
            if possible_names:
                for nameitem in possible_names: 
                    if len(nameitem)>1: 
                        updateOrCreateKey(all_comments, commenter, nameitem)
    final_cntr, counted_cmnts = superCount(comments, all_comments)
    nick_cntr = mergeByNicknames(final_cntr, celebs_nicknames)  
    #print (all_comments)
    print_results(nick_cntr, comments)
    return all_comments, comments, final_cntr, counted_cmnts, nick_cntr


if __name__ == "__main__":
    all_comments, comments, final_cntr, counted_cmnts, nick_cntr = main()



#getHealthReport(data, data_text, comments, all_comments)


Top guesses are:
Dean Cain (70%)
Superman (18%)


In [5]:
####///// HEALTH REPORT ////####

def getHealthReport(data, data_text, comments, all_comments):
    cmnts_on_site = data_text.count("<span class=\"says\">")
    cmnts_processed = 0
    for i in comments.values():
        for ii in i:
            cmnts_processed+=1
    cmnts_processed_names = len(comments)
    cmntrs_guessed = len(all_comments)
    cmnts_guessed = 0
    for i in all_comments.values():
        for ii in i:
            cmnts_guessed+=1        
    proc_ratio = round(cmnts_processed/cmnts_on_site*100)
    got_names_ratio = round(cmnts_processed_names/cmnts_processed*100)
    print ('\033[4mGeneral:\033[0m')
    print('{:.0f}% of the comments were processed'.format(proc_ratio))
    print('{}% of the processed comments yielded a guess'.format(got_names_ratio))
    print('Number of comments:',cmnts_on_site)
    names = []    
    for v in all_comments.values():
        for vv in v:
            names.append(vv)
    print ('Number of names in the comments:', len(names))        
    print ('Number of unique names in the comments:', len(set(names)))
    
    print( )
    print('\033[4mMore details:\033[0m')
    print ('processed {}/{} comments on the site, from {} commenters'.format(cmnts_processed, 
                                                                             cmnts_on_site, cmnts_processed_names))
    print ('Was able to get {1} guesses from {0} commenters that made guesses'.format(cmntrs_guessed, cmnts_guessed))
    
    print()
    # top names before the superCount
    print('\033[4mThese are the Top5 name counts before superCount:\033[0m')
    counter = healthCounter(all_comments)
    sorted_cntr = sorted(counter.items(), key = lambda counter: counter[1], reverse = True)
    for i in sorted_cntr[:5]:
        print(i[0]+":", i[1])
    print()
    
    # comments without guesses
    print ('\033[4mThese are the comments without names:\033[0m')
    for k, v in comments.items():
        if k not in all_comments:
            for vv in v:
                print ("> "+k+":",vv)
    print()
    
    # comments without guesses after superCount
    print ('\033[4mThese are the comments without names after super count:\033[0m')
    for k, v in comments.items():
        if k not in counted_cmnts:
            for item in v:
                print ("> "+k+":",item)
    print()
    
    # names guessed by a single person
    print ('\033[4mThese are the names that were guessed by a single person:\033[0m')
    for i in names:
        cntr = names.count(i)
        if cntr ==1:
            print(i)
    print()
    #short/long names
    print ('\033[4mUnusually long/short guesses:\033[0m')
    for i in names:
        if len(i)<=2 or len(i)>18:
            print(i)
    # cardinality
    uniqeness = round(len(set(names))/len(names)*100,2)
    if uniqeness>80:
        print()
        print ('\033[4mOops! cardinality is high:\033[0m', str(uniqeness)+'%', 'unique names/names ratio')
    print()
    # top commenter= highest number of guessed names
    sorted_dict = sorted(all_comments.items(), key= lambda x: len(x[1]), reverse=True)
    top = sorted_dict[0]
    if len(top[1])>1:
        top_cmntr = top[0]
        top_cmnts = top[1]
        print('\033[4mTop commenter is {}, with {} guesses:\033[0m'.format(top_cmntr, len(top_cmnts)))
        for i in top_cmnts:
            print(i)

getHealthReport(data, data_text, comments, all_comments)

General:
100% of the comments were processed
98% of the processed comments yielded a guess
Number of comments: 92
Number of names in the comments: 84
Number of unique names in the comments: 16

More details:
processed 92/92 comments on the site, from 90 commenters
Was able to get 84 guesses from 83 commenters that made guesses

These are the Top5 name counts before superCount:
dean cain: 69
dean cane: 1
stephen siegel: 1
pacey: 1
chris pratt: 1

These are the comments without names:
> GirlPlease: Superman Dean Cain of the 90s show Lois and Clark.
> melrose71: Dean Cain clues ‘90s series‘ and ‘flying high‘ for Lois and Clark Superman show. He looks fine. Honestly, it could be a bad angle and he could still get back in excellent shape in two months if he wanted. He’s in his 50s now, though, and maybe not chasing his youth like other actors. Maybe he has other priorities. Kudos to him.
> BeaNandSprouT: Dean Cain from Lois & Clark: The New Adventures of Superman.. Clues: flying high, Super

In [ ]:
### NEST STEPS ###
# For each guessed name, run the nicknames functions on the first and last name separately, to merge the count
# Fix code according to comments without guesses
# Extraction logic to implement: from the *entire* comment, grab the 1-2 upper words surrounded by lower case
# Extraction logic to implement: UPPER UPPER 'and'/& UPPER UPPER (2 names with "and" or "or" or "&" inbetween)
# Extend list of bad names to be removed
# Maybe extend cleaning logic in cleanBeforeAdding (re.sub(r'[^a-zA-Z1-9 ]', numbers, etc.)
# getUpperCaseFromShort- handle single-named celebs better, could be missed (sentance w/ only first word capitalized)
# Consider using nicknames, last names etc. as input for the superCount (even if they were not identified before SC)
# current nicknames logic is directional (k:v), can't find name by nickname. Consider changing? 
# Add to health report- tests related to the supercout (vs. original count) and to nick_cntr (vs. after superCount)